In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

tf.test.is_gpu_available()

2.0.0-dev20191002
sys.version_info(major=3, minor=7, micro=4, releaselevel='final', serial=0)
matplotlib 3.1.1
numpy 1.17.2
pandas 0.25.1
sklearn 0.21.3
tensorflow 2.0.0-dev20191002
tensorflow_core.keras 2.2.4-tf
Instructions for updating:
Use `tf.config.experimental.list_physical_devices('GPU')` instead.


False

# 数据类型
## 常量

In [5]:
# 常量
t = tf.constant([
                 [1.,2.,3.],
                 [4.,5.,6.]
                ])
# index
print(t)
print(t[:,1:])
print(t[...,1]) # 取所有第一维中第二维的index为1的值
print(t[1,...]) # 取第一维index为1的所有第二维的值

tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[2. 3.]
 [5. 6.]], shape=(2, 2), dtype=float32)
tf.Tensor([2. 5.], shape=(2,), dtype=float32)
tf.Tensor([4. 5. 6.], shape=(3,), dtype=float32)


In [6]:
# 操作opt
print(t+10) # 矩阵中所有的值+10
print(tf.square(t))
print(t @ tf.transpose(t)) # t与t的转置的矩阵乘法

tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[ 1.  4.  9.]
 [16. 25. 36.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[14. 32.]
 [32. 77.]], shape=(2, 2), dtype=float32)


In [8]:
# numpy conversion(与numpy之间的互相转换)
print(t.numpy()) # tensor转换成numpy
print(np.square(t)) # 对转换成numpy的矩阵进行运算
np_t = np.array([
                 [1.,2.,3.],
                 [4.,5.,6.]
                ])
print(tf.constant(np_t)) # 将numpy转换成tensor对象

[[1. 2. 3.]
 [4. 5. 6.]]
[[ 1.  4.  9.]
 [16. 25. 36.]]
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float64)


In [10]:
# 0维的就是一个数字，对应于TensorFlow中的Scalars
t = tf.constant(2.718)
print(t)
print(t.numpy())
print(t.shape)

tf.Tensor(2.718, shape=(), dtype=float32)
2.718
()


# 字符串

In [11]:
# strings
t = tf.constant('cafe')
print(t)
print(tf.strings.length(t))
print(tf.strings.length(t,unit="UTF8_CHAR")) # utf8的长度
print(tf.strings.unicode_decode(t,'UTF8')) # utf8的输出

tf.Tensor(b'cafe', shape=(), dtype=string)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor([ 99  97 102 101], shape=(4,), dtype=int32)


In [12]:
# string array
t = tf.constant(['cafe','coffee','咖啡'])
print(tf.strings.length(t,unit='UTF8_CHAR')) # 打印出数组中每一个字符串的utf8的长度
r = tf.strings.unicode_decode(t,'UTF8')
print(r) # RaggedTensor:不规则的tensor，统一维度的数字是不等长的 

tf.Tensor([4 6 2], shape=(3,), dtype=int32)
<tf.RaggedTensor [[99, 97, 102, 101], [99, 111, 102, 102, 101, 101], [21654, 21857]]>


# RaggedTensor 不规则的Tensor

In [13]:
# ragged tensor
r = tf.ragged.constant([
    [11,12],
    [21,22,23],
    [],
    [41],
])
print(r)
print(r[1]) # 只取第一行的就是一个一维的tensor
print(r[1:2]) # 左闭右开，但是返回的是一个RaggedTensor

<tf.RaggedTensor [[11, 12], [21, 22, 23], [], [41]]>
tf.Tensor([21 22 23], shape=(3,), dtype=int32)
<tf.RaggedTensor [[21, 22, 23]]>


In [15]:
# ops on ragged tensor :操作不规则tensor
r2 = tf.ragged.constant([[51,52],[],[71]])
print(tf.concat([r,r2],axis = 0)) # 在第0维度进行拼接

<tf.RaggedTensor [[11, 12], [21, 22, 23], [], [41], [51, 52], [], [71]]>


In [18]:
# print(tf.concat([r,r2],axis = 1)) # 在第一维进行拼接时 必须保证第0维的行数一致
r3 = tf.ragged.constant([[13,14],[15],[],[41,42]])
print(tf.concat([r,r3],axis = 1)) # 在第0维一致的情况下是可以进行第一维的拼接的

<tf.RaggedTensor [[11, 12, 13, 14], [21, 22, 23, 15], [], [41, 41, 42]]>


In [21]:
print(r.to_tensor()) # 将ragged tensor 变成普通的tensor，空位置默认用0补齐
print(r.to_tensor(default_value = 1)) # 将空位置用默认值来补齐

tf.Tensor(
[[11 12  0]
 [21 22 23]
 [ 0  0  0]
 [41  0  0]], shape=(4, 3), dtype=int32)
tf.Tensor(
[[11 12  1]
 [21 22 23]
 [ 1  1  1]
 [41  1  1]], shape=(4, 3), dtype=int32)


# Sparse Tensor ： 稀疏的Tensor

In [24]:
s = tf.SparseTensor(indices = [[0,1],[1,0],[2,3]], # 有值的位置索引，必须排好序，不然在to_dense会有问题
                    values = [1.,2.,3.], # 对应位置的值
                    dense_shape = [3,4] # 密集矩阵的shape
                   )
print(s)
print(tf.sparse.to_dense(s)) # 转换成密集矩阵

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
tf.Tensor(
[[0. 1. 0. 0.]
 [2. 0. 0. 0.]
 [0. 0. 0. 3.]], shape=(3, 4), dtype=float32)


In [26]:
# ops on spase tensors
s2 = s * 2.0
print(s2) # 乘法运算会将对应元素的位置进行乘法运算

try:
    s3 = s + 1 # spase tensor是不支持加法运算的
except TypeError as ex:
    print(ex)

s4 = tf.constant([[10.,20.],
                  [30.,40.],
                  [50.,60.],
                  [70.,80.]
                 ])
print(tf.sparse.sparse_dense_matmul(s,s4)) # sparse tensor做矩阵乘法

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([2. 4. 6.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
unsupported operand type(s) for +: 'SparseTensor' and 'int'
tf.Tensor(
[[ 30.  40.]
 [ 20.  40.]
 [210. 240.]], shape=(3, 2), dtype=float32)


In [28]:
s_no_order = tf.SparseTensor(indices = [[0,2],[0,1],[2,3]], # 测试如果不排好序
                             values = [1.,2.,3.],
                            dense_shape = [3,4])
print(s_no_order)
# print(tf.sparse.to_dense(s_no_order)) # 这种未排序的是不能to_dense的
s_order = tf.sparse.reorder(s_no_order) # 调用这个方法排下序列
print(tf.sparse.to_dense(s_order))

SparseTensor(indices=tf.Tensor(
[[0 2]
 [0 1]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
tf.Tensor(
[[0. 2. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 3.]], shape=(3, 4), dtype=float32)


# 变量 Variables

In [33]:
v = tf.Variable([[1.,2.,3.],
                 [4.,5.,6.]
                ])
print(v)
print(v.value())
print(v[:,1])
print(v[1,:])
print(v.numpy())
# 和常量的取值操作完全一致

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
tf.Tensor([2. 5.], shape=(2,), dtype=float32)
tf.Tensor([4. 5. 6.], shape=(3,), dtype=float32)
[[1. 2. 3.]
 [4. 5. 6.]]


In [39]:
# assign value：对变量重新赋值,只用assign来赋值，不能用=号赋值
v.assign(2*v)
print("对整个Variable赋值：\n",v.numpy())
v[0,1].assign(42)
print("\n对某个0维度的索引位置赋值：\n",v.numpy())
v[1].assign([7.,8.,9.])
print("\n对某个维度的一组索引位置赋值：\n",v.numpy())

对整个Variable赋值：
 [[ 64.  84. 192.]
 [ 14.  16.  18.]]

对某个0维度的索引位置赋值：
 [[ 64.  42. 192.]
 [ 14.  16.  18.]]

对某个维度的一组索引位置赋值：
 [[ 64.  42. 192.]
 [  7.   8.   9.]]


In [40]:
# 只用assign来赋值，不能用=号赋值
try:
    v[1] = [4.,8,9,]
except TypeError as ex:
    print(ex)

'ResourceVariable' object does not support item assignment
